Simple beat crash


In [1]:
from pydub import AudioSegment
import os

def make_mixtape(folder, output="mixtape.mp3", crossfade_ms=5000):
    files = [f for f in os.listdir(folder) if f.endswith((".mp3", ".wav"))]
    files.sort()  # optional, or random.shuffle(files)
    
    mixtape = None
    for i, file in enumerate(files):
        song = AudioSegment.from_file(os.path.join(folder, file))
        song = song.set_channels(2).set_frame_rate(44100)  # standardize
        song = song.apply_gain(-3)  # reduce loudness slightly
        
        if mixtape is None:
            mixtape = song
        else:
            mixtape = mixtape.append(song, crossfade=crossfade_ms)
    
    mixtape.export(output, format="mp3")
    print(f"✅ Mixtape generated: {output}")

# Example usage
make_mixtape("mixtape")



✅ Mixtape generated: mixtape.mp3


advanced test wrost

In [2]:
from pydub import AudioSegment
import os, random

def smooth_mixtape(folder, output="mixtape.mp3", crossfade_ms=10000):
    files = [f for f in os.listdir(folder) if f.endswith((".mp3", ".wav"))]
    random.shuffle(files)  # shuffle playlist
    
    mixtape = None
    for i, file in enumerate(files):
        song = AudioSegment.from_file(os.path.join(folder, file))
        song = song.set_channels(2).set_frame_rate(44100)

        if mixtape is None:
            mixtape = song
        else:
            # create transition part
            overlap = min(crossfade_ms, len(song), len(mixtape))
            
            # apply filters gradually (simulate DJ cutoff effect)
            outgoing = mixtape[-overlap:].high_pass_filter(200)  # cut bass on old
            incoming = song[:overlap].low_pass_filter(4000)      # cut highs on new

            transition = outgoing.overlay(incoming)
            
            mixtape = mixtape[:-overlap] + transition + song[overlap:]
    
    mixtape.export(output, format="mp3")
    print(f"✅ Smooth mixtape generated: {output}")

# Example usage
smooth_mixtape("mixtape")


✅ Smooth mixtape generated: mixtape.mp3


pt 2


In [28]:
from pydub import AudioSegment
import os, random

def smooth_fade_mixtape(folder, output="mixtape2.mp3", transition_ms=6000):
    files = [f for f in os.listdir(folder) if f.endswith((".mp3", ".wav"))]
    random.shuffle(files)  # shuffle for variety
    
    mixtape = None
    for i, file in enumerate(files):
        song = AudioSegment.from_file(os.path.join(folder, file))
        song = song.set_channels(2).set_frame_rate(44100)
        
        if mixtape is None:
            mixtape = song
        else:
            # define overlap region
            overlap = min(transition_ms, len(song), len(mixtape))
            
            # outgoing: fade out + high-pass to remove bass
            outgoing = mixtape[-overlap:].fade_out(overlap).low_pass_filter(4000)
            
            # incoming: fade in + low-pass to smooth highs
            incoming = song[:overlap].fade_in(overlap).low_pass_filter(4000)
            
            # overlay transition
            transition = outgoing.overlay(incoming)
            
            mixtape = mixtape[:-overlap] + transition + song[overlap:]
    
    mixtape.export(output, format="mp3")
    print(f"✅ Smooth fade mixtape generated: {output}")

# Example usage
smooth_fade_mixtape("mixtape")


✅ Smooth fade mixtape generated: mixtape2.mp3


BEAT MATCHING

In [ ]:
import os
import numpy as np
import librosa
from pydub import AudioSegment
import random

def detect_tempo_and_beats(path):
    """Load audio with librosa and return tempo + beat times (in ms)."""
    y, sr = librosa.load(path, sr=44100, mono=True)
    tempo, beat_frames = librosa.beat.beat_track(y=y, sr=sr)
    tempo = float(tempo)  # ensure scalar
    beat_times = librosa.frames_to_time(beat_frames, sr=sr) * 1000  # ms
    return tempo, beat_times


def get_key(path):
    """Rough key detection via librosa chroma (returns major/minor estimate)."""
    y, sr = librosa.load(path, sr=44100)
    chroma = librosa.feature.chroma_stft(y=y, sr=sr)
    chroma_mean = chroma.mean(axis=1)
    pitch_class = chroma_mean.argmax()

    keys_major = ["C", "C#", "D", "D#", "E", "F",
                "F#", "G", "G#", "A", "A#", "B"]
    return keys_major[pitch_class]

def beat_synced_crossfade(outgoing, incoming, beat_times_out, beat_times_in, overlap_beats=4):
    """
    Crossfade outgoing and incoming track aligned to beats.
    - overlap_beats: how many beats to overlap
    """
    # Pick last beat of outgoing, first beat of incoming
    if len(beat_times_out) < overlap_beats or len(beat_times_in) < overlap_beats:
        overlap_ms = 5000  # fallback to 5 sec
    else:
        overlap_ms = int(min(beat_times_out[-1] - beat_times_out[-overlap_beats],
                            beat_times_in[overlap_beats-1] - beat_times_in[0]))
    
    outgoing_seg = outgoing[-overlap_ms:].fade_out(overlap_ms).low_pass_filter(1000)
    incoming_seg = incoming[:overlap_ms].fade_in(overlap_ms).low_pass_filter(3000)
    
    return outgoing_seg.overlay(incoming_seg), overlap_ms

def smart_mixtape(folder, output="mixtape.mp3"):
    files = [f for f in os.listdir(folder) if f.endswith((".mp3", ".wav"))]
    random.shuffle(files)

    mixtape = None
    for i, file in enumerate(files):
        path = os.path.join(folder, file)
        print(f"Processing: {file}")
        tempo, beats = detect_tempo_and_beats(path)
        key = get_key(path)

        song = AudioSegment.from_file(path).set_channels(2).set_frame_rate(44100)

        print(f"  Tempo: {tempo:.1f} BPM | Key: {key}")


        if mixtape is None:
            mixtape = song
            prev_beats = beats
        else:
            transition, overlap = beat_synced_crossfade(mixtape, song, prev_beats, beats)
            mixtape = mixtape[:-overlap] + transition + song[overlap:]
            prev_beats = beats

    mixtape.export(output, format="mp3")
    print(f"✅ Beat-synced mixtape generated: {output}")

# Example usage
smart_mixtape("songs_folder")


Processing: HUGEL x Topic x Arash feat. Daecolm - I Adore You (with J Balvin & Ellie Goulding) (Extended Mix).mp3


C:\Users\djadh\AppData\Local\Temp\ipykernel_21976\3931486280.py:11: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  tempo = float(tempo)  # ensure scalar


  Tempo: 120.2 BPM | Key: B
Processing: Maz, VXSION - Amana.mp3
  Tempo: 123.0 BPM | Key: C#
Processing: Darmon, Mili & Sam Haze - Lila.mp3
  Tempo: 123.0 BPM | Key: G
✅ Beat-synced mixtape generated: mixtape.mp3


### video integration

In [29]:
import subprocess
import os

def make_video_from_audio(image_path, audio_path, output_path="mixtape_vid.mp4"):
    """
    Merge a static image with audio to create a video.
    Resizes image to even dimensions (required by libx264).
    Re-encodes audio to AAC for MP4.
    """
    if not os.path.exists(image_path):
        raise FileNotFoundError(f"Image not found: {image_path}")
    if not os.path.exists(audio_path):
        raise FileNotFoundError(f"Audio not found: {audio_path}")

    cmd = f'ffmpeg -y -loop 1 -i "{image_path}" -i "{audio_path}" ' \
        f'-vf "scale=trunc(iw/2)*2:trunc(ih/2)*2" ' \
        f'-c:v libx264 -preset fast -tune stillimage ' \
        f'-c:a aac -b:a 192k -shortest "{output_path}"'

    try:
        subprocess.run(cmd, shell=True, check=True)
        print(f"✅ Video created: {output_path}")
    except subprocess.CalledProcessError:
        print("❌ FFmpeg failed! Please check image/audio paths and formats.")

# Example usage
image_path = r"ff.jpg"
audio_path = r"mixtape2.mp3"
make_video_from_audio(image_path, audio_path)


✅ Video created: mixtape_vid.mp4
